In [98]:
import pandas as pd
import numpy as np

def contains_numeric(input_string):
    for char in input_string:
        if char.isdigit():
            return True
    return False

def clean_list(lst):
    filtered = list()
    for i in lst:
        if "(" in i or "/" in i or i == "" or "." in i or contains_numeric(i):
            continue
        else:
            filtered.append(i)
    
    return filtered


def get_nested_elements_as_string(lst):
    result = ""
    for item in lst:
        if isinstance(item, list):
            result += get_nested_elements_as_string(item)
        else:
            if item:
                result += str(item).lower() + ";"
    return result.strip()


In [99]:
data = pd.read_csv("p2/data_hero.csv")

In [100]:
import pickle

with open("p1/hero.pkl", "rb") as t:
    heroes = pickle.load(t)

In [101]:
heroes = [i.lower() for i in heroes]

In [102]:
data.shape

(123, 2)

In [103]:
def filterize(inp):
    global heroes

    stringy = get_nested_elements_as_string(eval(inp))
    m = list(set(stringy.split(";")) - set(heroes))
    return clean_list(m)

In [104]:
data["cleaned"] = data["Skill"].apply(filterize)
data_clean = data[["Hero","cleaned"]]
data_clean

Hero                                            cleaned
0       Miya  [moon arrow, skill terms, movement speed, atta...
1    Balmond  [skill terms, blink, movement speed, physical ...
2      Saber  [skill terms, blink, triple sweep, physical da...
3      Alice  [skill terms, movement speed, blood ode, blink...
4       Nana  [skill terms, movement speed, debuffs, transfo...
..       ...                                                ...
118  Novaria  [skill terms, movement speed, vision, astral s...
119   Arlott  [physical damage, dauntless strike, mobility, ...
120     Ixia  [pulling, physical damage, critically strike, ...
121    Nolan  [skill terms, expansion, debuffs, dash, except...
122     Cici  [yo-yo blitz, buoyant bounce, burst, cc, perfo...

[123 rows x 2 columns]

In [105]:
all_traits = set(trait.replace(":","") for traits_list in data['cleaned'] for trait in traits_list)
all_traits_list = list(all_traits)
df = pd.DataFrame(np.hstack([data_clean.values,np.zeros((123,len(all_traits)))]))
df.columns = ["hero","traits"] + list(all_traits)

In [106]:
for col_nm in df.columns[2:]:
    df[col_nm] = df.apply(lambda row: 1 if col_nm in row["traits"] else np.nan, axis=1)

In [107]:
df.shape

(123, 993)

In [108]:
selected_columns = df.loc[:, (df == 1).sum() > 2].copy()

In [109]:
selected_columns["Hero"] = df["hero"].copy()

In [110]:
selected_columns.to_csv("kesimpulan.csv", index=False)

In [111]:
selected_columns

knocked back  magic damage  control immunity  the oriental fighters  \
0             NaN           NaN               NaN                    NaN   
1             1.0           NaN               NaN                    NaN   
2             NaN           NaN               NaN                    NaN   
3             NaN           1.0               NaN                    NaN   
4             NaN           1.0               NaN                    NaN   
..            ...           ...               ...                    ...   
118           NaN           1.0               NaN                    NaN   
119           NaN           NaN               NaN                    NaN   
120           NaN           NaN               NaN                    NaN   
121           NaN           NaN               NaN                    NaN   
122           NaN           NaN               NaN                    NaN   

     immune to control effects  charge  cooldown  mobility  crit damage  \
0                          NaN     NaN       1.0       NaN          NaN   
1                          NaN     NaN       1.0       1.0          NaN   
2                          NaN     1.0       1.0       1.0          NaN   
3                          NaN     NaN       1.0       1.0          NaN   
4                          NaN     NaN       1.0       NaN          NaN   
..                         ...     ...       ...       ...          ...   
118                        NaN     NaN       1.0       NaN          NaN   
119                        NaN     NaN       1.0       1.0          NaN   
120                        NaN     NaN       1.0       NaN          NaN   
121                        NaN     NaN       1.0       1.0          NaN   
122                        NaN     NaN       1.0       1.0          NaN   

     cooldowns  ...  conceal  damage  recharging time  dashes  teleports  \
0          NaN  ...      1.0     NaN              NaN     NaN        NaN   
1          NaN  ...      NaN     NaN              NaN     NaN        NaN   
2          NaN  ...      NaN     NaN              NaN     1.0        NaN   
3          NaN  ...      NaN     NaN              NaN     NaN        1.0   
4          NaN  ...      NaN     NaN              NaN     NaN        NaN   
..         ...  ...      ...     ...              ...     ...        ...   
118        NaN  ...      NaN     NaN              NaN     NaN        NaN   
119        NaN  ...      NaN     1.0              NaN     NaN        NaN   
120        NaN  ...      NaN     1.0              NaN     NaN        NaN   
121        NaN  ...      NaN     1.0              NaN     1.0        NaN   
122        NaN  ...      NaN     NaN              NaN     NaN        NaN   

     lifesteal   hp  slow immunity  atk speed     Hero  
0          NaN  NaN            NaN        NaN     Miya  
1          NaN  NaN            NaN        NaN  Balmond  
2          NaN  NaN            NaN        NaN    Saber  
3          NaN  1.0            NaN        NaN    Alice  
4          NaN  NaN            NaN        NaN     Nana  
..         ...  ...            ...        ...      ...  
118        NaN  NaN            NaN        NaN  Novaria  
119        NaN  1.0            NaN        NaN   Arlott  
120        1.0  1.0            NaN        NaN     Ixia  
121        NaN  NaN            NaN        NaN    Nolan  
122        NaN  NaN            NaN        NaN     Cici  

[123 rows x 129 columns]